# Workshop 3: TensorFlow - Detecting Handwritten Digits

This is a handwritten character image (MNIST) classifier that can run on any Android device. The app stores a model  set of images (0-9) that we can cycle through and classify in order. It uses a pre-trained model to perform inference on the device. This idea can be applied to any images, both by using the camera and by pulling from the Web. We're using preloaded images so we can run the app in a simulator (no need for the device since it doesn't require a camera).

* <a href="https://www.youtube.com/watch?v=gahi0Hjgokw">__Video Demo__</a>
* <a href="https://github.com/miyosuda/TensorFlowAndroidMNIST">__App Source Code__</a>

<img src="https://github.com/llSourcell/A_Guide_to_Running_Tensorflow_Models_on_Android/raw/master/images/demo.png" width="800">

This Android demo is based on Tensorflow tutorials:
* <a href="https://www.tensorflow.org/versions/r0.10/tutorials/mnist/beginners/index.html">MNIST For ML Beginners</a>
* <a href="https://www.tensorflow.org/versions/r0.10/tutorials/mnist/pros/index.html">Deep MNIST for Experts</a>



## 1. Building / Training / Testing a Model

### Data 

MNIST is a simple computer vision dataset (70'000 labeled examples). It consists of 28x28 pixel images of handwritten digits. Every MNIST data point, every image, can be thought of as an array of numbers describing how dark each pixel is. For example, we might think of 1 as something like:

<img src="https://tensorflow.rstudio.com/tensorflow/articles/images/MNIST-Matrix.png" width="600">

Since each image has 28 by 28 pixels, we get a 28x28 array. We can flatten each array into a 28∗28=784
 dimensional vector. Each component of the vector is a value between zero and one describing the intensity of the pixel. Thus, we generally think of MNIST as being a collection of 784-dimensional vectors.
 
The __y_train__ data is the associated labels for all the __x_train__ examples. Rather than storing the label as an integer, it is stored as a 1x10 binary array with the one representing the digit. This is also known as one-hot encoding. In the example below, the array represents a 7:

<img src="https://d3ansictanv2wj.cloudfront.net/Img-1-array-b4889b9860c9e009bbd58e827a114129.png" width="600">

In [1]:
# Python 3.6.0
# tensorflow 1.5.0
# Keras 2.0.4

import os
import os.path as path

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = load_data()

Using TensorFlow backend.


### Exploration

In [2]:
import matplotlib.pyplot as plt
import numpy as np

def display_digit(digit, title = ""):
    """
    graphically displays a 784x1 vector, representing a digit
    """
    plt.figure()
    fig = plt.imshow(digit.flatten().reshape(28,28))
    fig.set_cmap('gray_r')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    if title != "":
        plt.title("Inferred label: " + str(title))
        
digit = 129
display_digit(x_train[digit], title = y_train[digit])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  del sys.path[0]


### Building a Model

Read: <a href="https://hackernoon.com/visualizing-parts-of-convolutional-neural-networks-using-keras-and-cats-5cc01b214e59">Visualizing parts of Convolutional Neural Networks using Keras and Cats</a>

In [4]:
def build_model():
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=3, strides=1, \
            padding='same', activation='relu', \
            input_shape=[28, 28, 1]))
    # 28*28*64
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    
    # 14*14*64
    model.add(Conv2D(filters=128, kernel_size=3, strides=1, \
            padding='same', activation='relu'))

    # 14*14*128
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    
    # 7*7*128
    model.add(Conv2D(filters=256, kernel_size=3, strides=1, \
            padding='same', activation='relu'))
    
    # 7*7*256
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    
    # 4*4*256
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model

model = build_model()

### Training / Testing / Validation

When solving with a CPU an Optimization Problem, you Iteratively apply an Algorithm over some Input Data. In each of these iterations you usually update a Metric of your problem doing some Calculations on the Data. Now when the size of your data is large it might need a considerable amount of time to complete every iteration, and may consume a lot of resources. So sometimes you choose to apply these iterative calculations on a Portion of the Data to save time and computational resources. This portion is the batch_size and the process is called (in the Neural Network Lingo) batch data processing.

In the neural network terminology:
* one __epoch__ = one forward pass and one backward pass of all the training examples
* __batch size__ = the number of training examples in one forward/backward pass. The higher the batch size, the more memory space you'll need.
* number of __iterations__ = number of passes, each pass using [batch size] number of examples. To be clear, one pass = one forward pass + one backward pass (we do not count the forward pass and backward pass as two different passes).

Advantages:
* It requires less memory. Since you train network using less number of samples the overall training procedure requires less memory. It's especially important in case if you are not able to fit dataset in memory.
* Typically networks trains faster with mini-batches. That's because we update weights after each propagation. In our example we've propagated 11 batches (10 of them had 100 samples and 1 had 50 samples) and after each of them we've updated network's parameters. If we used all samples during propagation we would make only 1 update for the network's parameter.

Disadvantages:
* The smaller the batch the less accurate estimate of the gradient. In the figure below you can see that mini-batch (green color) gradient's direction fluctuates compare to the full batch (blue color).

<img src="https://i.stack.imgur.com/lU3sx.png" width="700">

Training code below takes around 5 minutes.

In [5]:
EPOCHS = 1
BATCH_SIZE = 128

model.compile(loss=keras.losses.categorical_crossentropy, \
    optimizer=keras.optimizers.Adadelta(), \
    metrics=['accuracy'])

model.fit(x_train, y_train, \
    batch_size=BATCH_SIZE, \
    epochs=EPOCHS, \
    verbose=1, \
    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 358s 6ms/step - loss: 0.6499 - acc: 0.7956 - val_loss: 0.1592 - val_acc: 0.9497


### Export

How to export my model?

1. Train your model

2. Keep an in memory copy of everything your model learned (like biases and weights) Example: `_w = sess.eval(w)`, where `w` was learned from training.

3. Rewrite your model changing the variables for constants with value = in memory copy of learned variables. Example:
`w_save = tf.constant(_w)`

4. Also make sure to put names in the input and output of the model, this will be needed for the model later. Example:

`x = tf.placeholder(tf.float32, [None, 1000], name='input')
y = tf.nn.softmax(tf.matmul(x, w_save) + b_save), name='output')`

5. Export your model with:
tf.train.write_graph(<graph>, <path for the exported model>, <name of the model>.pb, as_text=False)

In [8]:
MODEL_NAME = 'mnist_convnet'
DIR_NAME = 'ws3_hwd_data/'

def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, DIR_NAME, \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), DIR_NAME + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph(DIR_NAME + MODEL_NAME + '_graph.pbtxt', None, \
        False, DIR_NAME + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        DIR_NAME + 'frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open(DIR_NAME + 'frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile(DIR_NAME + 'opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")


if not path.exists(DIR_NAME):
    os.mkdir(DIR_NAME)

    export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Softmax")

INFO:tensorflow:Restoring parameters from ws3_hwd_data/mnist_convnet.chkp
INFO:tensorflow:Froze 10 variables.
Converted 10 variables to const ops.
graph saved!


***
## 2. Android App

NDK (Native Development Kit) is a tool that allows you to program in C/C++ for Android devices. It integrates with the SDK and is used for performance-critical code. You should install NDK support in your Android Studio.

* <a href="https://developer.android.com/ndk/downloads/">Download and install Android NDK</a>

![alt text](https://jalammar.github.io/images/android-tensorflow-app-structure_2.png)

* Examine and run on your hardware <a href="https://github.com/miyosuda/TensorFlowAndroidMNIST">app source code</a>


***
## Related Examples and Useful Links

* <a href="https://www.oreilly.com/learning/not-another-mnist-tutorial-with-tensorflow">Not another MNIST tutorial with TensorFlow</a>


***
## Credits
* YouTube video by Siraj Raval: <a href="https://www.youtube.com/watch?v=kFWKdLOxykE&feature=youtu.be">A Guide to Running Tensorflow Models on Android</a>
* <a href="https://github.com/llSourcell/A_Guide_to_Running_Tensorflow_Models_on_Android">App source code</a> adjusted by Siraj Raval
* Original <a href="https://github.com/miyosuda/TensorFlowAndroidMNIST">app source code</a>